# **Notebook Setup**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.1 MB/s eta 0:00:00


# **Imports**

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

# **Model and Tokenizer**

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

# **Data Loading**

In [5]:
drive_path = 'drive/MyDrive/MetalliA/'# dataset/ models/

In [6]:
def parse_lyrics(root, song)-> str:
  '''
    Removes title and empty rows from song
    Returns lyrics without linebreaks
  '''
  if song.__contains__('.txt'):
    with open(os.path.join(root, song), "r") as file:
      lyrics = file.read()
      lyrics = [line+' <|endofverse|>' for line in lyrics.split('\n') if len(line) > 0]
      lyrics = ' '.join(lyrics)
  return lyrics

**Data example for clarification**

root: drive/MyDrive/MetalliA/dataset/5-black_album

song: dont_tread_on_me.txt

In [7]:
data_path = drive_path + 'dataset/'

df = pd.DataFrame(columns=['album', 'song_name', 'lyrics'])

for root, dirs, files in os.walk(data_path):
  for song in files:
    #extract album name from path
    album = root.split('/')[-1]
    #remove album number from name
    album = album.split('-')[-1]

    #parse song name
    song_name = song.split('.')[0]
    song_name = ' '.join(song_name.split('_'))

    lyrics = parse_lyrics(root, song)

    row = {'album': album, 'song_name': song_name, 'lyrics': lyrics}
    df = df.append(row, ignore_index=True)


<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [8]:
print("total of songs: {}".format(len(df.index)))

total of songs: 106


In [9]:
df.head(10)

,album,song_name,lyrics
0,black_album,dont tread on me,"""Don't Tread On Me"" <|endofverse|> Liberty or ..."
1,black_album,of wolf and man,"""Of Wolf And Man"" <|endofverse|> Off through t..."
2,black_album,wherever i may roam,"""Wherever I May Roam"" <|endofverse|> ...and th..."
3,black_album,holier than thou,"""Holier Than Thou"" <|endofverse|> No more! <|e..."
4,black_album,the god that failed,"""The God That Failed"" <|endofverse|> Pride you..."
5,black_album,nothing else matters,"""Nothing Else Matters"" <|endofverse|> So close..."
6,black_album,the unforgiven,"""The Unforgiven"" <|endofverse|> New blood join..."
7,black_album,my friend of misery,"""My Friend Of Misery"" <|endofverse|> You just ..."
8,black_album,enter sandman,"""Enter Sandman"" <|endofverse|> Say your prayer..."
9,black_album,through the never,"""Through The Never"" <|endofverse|> All that is..."


In [10]:
lir = list(df['lyrics'])
song_length = [len(s.split(' ')) for s in lir]
print(song_length)

[246, 258, 312, 212, 325, 263, 379, 244, 260, 244, 371, 256, 216, 305, 314, 260, 234, 170, 324, 280, 229, 312, 134, 229, 246, 384, 413, 253, 279, 235, 358, 206, 376, 368, 286, 213, 210, 241, 346, 327, 193, 357, 361, 302, 368, 285, 450, 335, 332, 235, 298, 254, 471, 264, 326, 249, 419, 433, 279, 191, 287, 270, 238, 152, 236, 179, 441, 409, 207, 266, 190, 258, 248, 276, 354, 217, 244, 199, 217, 312, 262, 213, 243, 283, 138, 347, 183, 237, 499, 275, 308, 599, 509, 161, 404, 272, 501, 224, 291, 310, 389, 481, 272, 348, 326, 322]


We split the data into train and test set.

In [11]:
test_set = df.sample(n = 10)
train_set = df.loc[~df.index.isin(test_set.index)]
print('train set size: {}'.format(len(train_set.index)))
print('test set size: {}'.format(len(test_set.index)))

train set size: 96
test set size: 10


In [12]:
test_set = test_set.reset_index()
train_set = train_set.reset_index()

In the test set, we remove the last part of the songs in order to measure the accuracy of our trained model

In [13]:
test_set['true_lyrics'] = test_set['lyrics'].str.split().str[-20:].apply(' '.join)
test_set['lyrics'] = test_set['lyrics'].str.split().str[:-20].apply(' '.join)

In [14]:
test_set.head(10)

,index,album,song_name,lyrics,true_lyrics
0,95,st_anger,my world,"""My World"" <|endofverse|> The motherfuckers go...","head <|endofverse|> Get 'em out of my head, ou..."
1,91,st_anger,st anger,"""St. Anger"" <|endofverse|> Saint Anger 'round ...",madly in anger with you <|endofverse|> I'm mad...
2,69,master_of_puppets,damage inc,"""Damage, Inc."" <|endofverse|> Dealing out the ...",a little closer if you please <|endofverse|> B...
3,35,load,wasting my hate,"""Wasting My Hate"" <|endofverse|> Good day. How...",won’t waste my hate on you <|endofverse|> Wast...
4,30,and_justice_for_all,and justice for all,"""...And Justice For All"" <|endofverse|> Halls ...",<|endofverse|> Seeking no truth <|endofverse|>...
5,76,hardwired_to_self_destruct,halo on fire,"""Halo On Fire"" <|endofverse|> Obey <|endofvers...",stay <|endofverse|> Hello darkness <|endofvers...
6,39,load,2 x 4,"""2 x 4"" <|endofverse|> I’m gonna make you, sha...","meet my lord <|endofverse|> I can’t hear you, ..."
7,2,black_album,wherever i may roam,"""Wherever I May Roam"" <|endofverse|> ...and th...",may roam <|endofverse|> Wherever I may roam <|...
8,49,reload,better than you,"""Better Than You"" <|endofverse|> I look at you...","on and on <|endofverse|> Forever on and on, on..."
9,90,st_anger,dirty window,"""Dirty Window"" <|endofverse|> I see my reflect...",my throne <|endofverse|> I drink from the cup ...


Create the dataset acording to GPT-2 expected format. 
Song max lenth is limited to 1024 tokens since GPT-2 performance gets worse from that length on.

We have to wrap it up into a class that inherits from torch Dataset so we can enumerate and loop over it.

In [15]:
class Lyrics(Dataset):
  def __init__(self, data, max_len = 1024):
    self.tokenizer = tokenizer
    self.lyrics = []

    for row in data['lyrics']:
      self.lyrics.append(
          torch.tensor(
            tokenizer.encode(f"<|[lyrics]|>{row[:max_len]}<|endoftext|>")
          )
      )
    
    self.count = len(self.lyrics)

  def __len__(self):
    return self.count

  def __getitem__(self, item):
    return self.lyrics[item]

In [16]:
tr_dataset = Lyrics(train_set)

# **Model Training**

First we define the hyper-parameters for our model

In [17]:
BATCH_SIZE = 10
EPOCHS = 50
LEARNING_RATE = 1e-4
WARM_STEPS = 200

models_path = drive_path + 'models/'

In [18]:
def train(
    dataset, output_prefix="MetalliA", save_model_on_epoch=True,
):
  gpt2.train()

  optimizer = AdamW(gpt2.parameters(), lr=LEARNING_RATE)
  scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=WARM_STEPS, num_training_steps=-1
  )

  train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
  loss=0
  accumulating_batch_count = 0
  input_tensor = None
  min_loss = float("Inf")
  for epoch in range(EPOCHS):

    print(f"Training epoch {epoch}")
    
    for idx, entry in tqdm(enumerate(train_dataloader)):

      input_tensor = entry.to(device)
      outputs = gpt2(input_tensor, labels=input_tensor)
      loss = outputs[0]
      loss.backward()

      if (accumulating_batch_count % BATCH_SIZE) == 0:
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        gpt2.zero_grad()

      accumulating_batch_count += 1
      
    if save_model_on_epoch:
        if loss < min_loss:
          torch.save(
              gpt2.state_dict(),
              os.path.join(models_path, f"{output_prefix}-{epoch}.pt"),
          )
          min_loss = loss
    print(loss)
    print()
        
  return gpt2

In [19]:
device=torch.device("cuda")
gpt2 = gpt2.cuda()
gpt2_finetune = train(tr_dataset)

Training epoch 0


96it [00:14,  6.76it/s]


tensor(1.7527, device='cuda:0', grad_fn=<NllLossBackward0>)

Training epoch 1


96it [00:11,  8.54it/s]


tensor(2.7423, device='cuda:0', grad_fn=<NllLossBackward0>)

Training epoch 2


...

tensor(0.6819, device='cuda:0', grad_fn=<NllLossBackward0>)

Training epoch 48


96it [00:11,  8.18it/s]


tensor(0.9914, device='cuda:0', grad_fn=<NllLossBackward0>)

Training epoch 49


96it [00:11,  8.17it/s]

tensor(0.7219, device='cuda:0', grad_fn=<NllLossBackward0>)



In [20]:
def generate(input, max_len = 30, top_p = 0.8, temp=1.0):
  #model_cpu = model.to('cpu')
  gpt2_finetune.eval()

  device = torch.device("cuda")
  filter_value = -float("Inf")

  new_only = torch.tensor([]).unsqueeze(0).to(device)

  with torch.no_grad():
    generation = torch.tensor(tokenizer.encode(input)).unsqueeze(0).to(device)

    for i in tqdm(range(max_len)):
      outputs = gpt2_finetune(generation, labels = generation)
      loss, logits = outputs[:2]
      logits = logits[:, -1, :] / (temp if temp > 0 else 1.0)

      sorted_logits, sorted_index = torch.sort(logits, descending=True)
      cum_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim = -1)

      sorted_indices_to_remove = cum_probs > top_p
      sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
          ..., :-1
      ].clone()
      sorted_indices_to_remove[..., 0] = 0

      indices_to_remove = sorted_index[sorted_indices_to_remove]
      logits[:, indices_to_remove] = filter_value

      next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)

      if next_token in tokenizer.encode("<|endoftext|>"):
        break 

      new_only = torch.cat((new_only, next_token), dim=1)
      generation = torch.cat((generation, next_token), dim=1)
    
    output_list = list(generation.squeeze().cpu().numpy())
    output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 

    only_gen_list = list(new_only.squeeze().cpu().numpy()) if not new_only.size(dim=-1) == 0 else []
    only_gen = f"{tokenizer.decode(only_gen_list)}<|endoftext|>" if not len(only_gen_list) == 0 else f"<|endoftext|>"

  return output_text, only_gen

In [21]:
def text_generation(test_data):
  generated_lyrics = []
  full_lyrics = []
  for i in range(len(test_data)):
    full_song, gen_verses = generate(test_data['lyrics'][i])
    generated_lyrics.append(gen_verses)
    full_lyrics.append(full_song)
  return generated_lyrics, full_lyrics

In [23]:
generated_lyrics, full_lyrics = text_generation(test_set)

  0%|          | 0/30 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
generated_lyrics

In [ ]:
test_set['Generated_lyrics'] = generated_lyrics

In [ ]:
test_set.head(10)

In [ ]:
for i in range(len(test_set)):
  print("song name:{}".format(test_set['song_name'][i]))
  print("TRUE lyrics:\t\t{}".format(test_set['true_lyrics'][i]))
  print("Model generation:\t{}".format(test_set['Generated_lyrics'][i]))
  
  print()

In [ ]:
title = '"One, two, three... die after me" <|endofverse|>'
full_song, _ = generate(title, max_len=350, temp = 1.15)

In [ ]:
print(full_song.replace('<|endofverse|>', '\n'))
